In [15]:
import sys
# Update the path to point to the correct location of the 'mechsystem' folder
sys.path.append('../mechsystem')

from mass_spring import *  # Import the mass-spring system module

from pythreejs import *  # Import pythreejs for 3D visualization


In [16]:
# Import necessary classes from mass_spring
from mass_spring import Mass, MassSpringSystem3d, Fix, Spring, DistanceConstraint

# Create an instance of the mass-spring system
mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)  # Set gravity (in the z-direction)

# Define the three masses for the spinning top (in 3D)
mA = mss.add(Mass(1.0, [1.0, 0.0, 0.0]))  # Mass 1 at (1, 0, 0)
mB = mss.add(Mass(1.0, [-1.0, 0.0, 0.0]))  # Mass 2 at (-1, 0, 0)
mC = mss.add(Mass(1.0, [0.0, 1.0, 0.0]))  # Mass 3 at (0, 1, 0)

# Check if the objects are added correctly
print("Object mA:", mA)  # Should not be None
print("Object mB:", mB)  # Should not be None
print("Object mC:", mC)  # Should not be None

# Set initial velocities for rotational motion (angular velocity around z-axis)
if mA is not None: mA.vel = [0.0, 0.0, 1.0]  # Rotational velocity for mA
if mB is not None: mB.vel = [0.0, 0.0, 1.0]  # Rotational velocity for mB
if mC is not None: mC.vel = [0.0, 0.0, 1.0]  # Rotational velocity for mC

# Define a fixed point (can be placed at the center)
f1 = mss.add(Fix([0.0, 0.0, 0.0]))  # Fix at origin

# Add springs (connecting the masses to a fixed point)
mss.add(Spring(1.0, 200000, (f1, mA)))  # Spring between fix and mA
mss.add(Spring(1.0, 200000, (f1, mB)))  # Spring between fix and mB
mss.add(Spring(1.0, 200000, (f1, mC)))  # Spring between fix and mC

# Add distance constraints to keep the masses at fixed distances from each other
L1 = 1.0  # Distance between mA and mB
L2 = 1.0  # Distance between mB and mC
L3 = 1.0  # Distance between mC and mA

# Add distance constraints
dc1 = DistanceConstraint(mA, mB, L1)
dc2 = DistanceConstraint(mB, mC, L2)
dc3 = DistanceConstraint(mC, mA, L3)

mss.addDistanceConstraint(dc1)
mss.addDistanceConstraint(dc2)
mss.addDistanceConstraint(dc3)


Adding object of type: <class 'mass_spring.Mass'>
Added Mass: Mass(mass=1.0, pos=[1.0, 0.0, 0.0], vel=[0.0, 0.0, 0.0])
Adding object of type: <class 'mass_spring.Mass'>
Added Mass: Mass(mass=1.0, pos=[-1.0, 0.0, 0.0], vel=[0.0, 0.0, 0.0])
Adding object of type: <class 'mass_spring.Mass'>
Added Mass: Mass(mass=1.0, pos=[0.0, 1.0, 0.0], vel=[0.0, 0.0, 0.0])
Object mA: Mass(mass=1.0, pos=[1.0, 0.0, 0.0], vel=[0.0, 0.0, 0.0])
Object mB: Mass(mass=1.0, pos=[-1.0, 0.0, 0.0], vel=[0.0, 0.0, 0.0])
Object mC: Mass(mass=1.0, pos=[0.0, 1.0, 0.0], vel=[0.0, 0.0, 0.0])
Adding object of type: <class 'mass_spring.Fix'>
Added Fix: Fix(pos=[0.0, 0.0, 0.0])
Adding object of type: <class 'mass_spring.Spring'>
Added Spring: Spring(length=1.0, stiffness=200000, connectors=(Fix(pos=[0.0, 0.0, 0.0]), Mass(mass=1.0, pos=[1.0, 0.0, 0.0], vel=[0.0, 0.0, 1.0])))
Adding object of type: <class 'mass_spring.Spring'>
Added Spring: Spring(length=1.0, stiffness=200000, connectors=(Fix(pos=[0.0, 0.0, 0.0]), Mass(mass=1

In [17]:
m1 = mss.add(Mass(1.0, [0, 0, 0]))  # Example mass 1 at origin
m2 = mss.add(Mass(1.0, [1, 0, 0]))  # Example mass 2 at (1, 0, 0)

# Set distance constraint between m1 and m2 (e.g., distance is 1.0)
dc = DistanceConstraint(m1, m2, 1.0)
mss.addDistanceConstraint(dc)

print(len(mss.constraints))  # Output the number of constraints


Adding object of type: <class 'mass_spring.Mass'>
Added Mass: Mass(mass=1.0, pos=[0, 0, 0], vel=[0.0, 0.0, 0.0])
Adding object of type: <class 'mass_spring.Mass'>
Added Mass: Mass(mass=1.0, pos=[1, 0, 0], vel=[0.0, 0.0, 0.0])
Added DistanceConstraint: DistanceConstraint(c1=Mass(mass=1.0, pos=[0, 0, 0], vel=[0.0, 0.0, 0.0]), c2=Mass(mass=1.0, pos=[1, 0, 0], vel=[0.0, 0.0, 0.0]), rest_length=1.0)
4


In [27]:
# Create meshes for masses
from pythreejs import Mesh, SphereBufferGeometry, MeshStandardMaterial, AxesHelper, LineSegments, LineSegmentsGeometry, LineMaterial, LineSegments2, Scene, PerspectiveCamera, DirectionalLight, AmbientLight, Renderer, OrbitControls

masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),  # Radius 0.2, 16 segments
             MeshStandardMaterial(color='red'),
             position=m.pos))  # Set position of each mass

# Create meshes for fixes (if any)
fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos))  # Set position of each fix

# Create spring connections (using lines)
springpos = []
for s in mss.springs:
    # Handling connectors as either Mass or Fix
    if isinstance(s.connectors[0], Mass):  # If the connector is a Mass
        pA = s.connectors[0].pos  # Access the position of the Mass object
    else:  # If the connector is a Fix
        pA = s.connectors[0].pos  # Access the position of Fix
    
    if isinstance(s.connectors[1], Mass):  # If the connector is a Mass
        pB = s.connectors[1].pos  # Access the position of the Mass object
    else:  # If the connector is a Fix
        pB = s.connectors[1].pos  # Access the position of Fix
    
    springpos.append([pA, pB])  # Add spring start and end positions to the list

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')  # Line material with cyan color
springs = LineSegments2(springgeo, m2)  # Creating a LineSegments2 object to represent the springs

# Create axes for reference
axes = AxesHelper(1)  # Creates axes with size 1 for reference in 3D space

# Add all objects to the scene
scene = Scene(children=[*masses, *fixes, springs, axes])

# Setup the camera and lighting
camera = PerspectiveCamera(position=[10, 6, 10], aspect=1.0)  # Camera position
key_light = DirectionalLight(position=[0, 10, 10])  # Key light for illumination
ambient_light = AmbientLight()  # Ambient light for scene

# Add camera, light, and scene to the renderer
controller = OrbitControls(controlling=camera)  # Controls for moving the camera
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=600, height=400)

# Display the 3D scene in Jupyter
renderer


Renderer(camera=PerspectiveCamera(position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, …

In [36]:
from pythreejs import Mesh, SphereBufferGeometry, MeshStandardMaterial, AxesHelper, LineSegments, LineSegmentsGeometry, LineMaterial, LineSegments2, Scene, PerspectiveCamera, DirectionalLight, AmbientLight, Renderer, OrbitControls
import numpy as np
from time import sleep

# Set initial angular velocity (spin along Z-axis)
initial_angular_velocity = [0, 0, 10]  # Spin around Z-axis
angular_velocity = np.array(initial_angular_velocity)

# Function to apply angular velocity and simulate rotation around the Z-axis
def apply_angular_velocity(masses, angular_velocity, dt):
    for m in masses:
        # Use rotational transformation matrix for spinning along Z-axis
        rotation_matrix = np.array([
            [np.cos(angular_velocity[2]*dt), -np.sin(angular_velocity[2]*dt), 0],
            [np.sin(angular_velocity[2]*dt), np.cos(angular_velocity[2]*dt), 0],
            [0, 0, 1]
        ])
        m.pos = np.dot(rotation_matrix, m.pos)  # Apply rotation to the mass position

# Create meshes for masses (representing the top's body)
masses = []
for m in mss.masses:
    # Ensure m.pos is a list or numpy array with 3 elements (x, y, z)
    if len(m.pos) == 3:
        masses.append(
            Mesh(SphereBufferGeometry(0.2, 16, 16),  # Radius 0.2, 16 segments
                 MeshStandardMaterial(color='red'),
                 position=[m.pos[0], m.pos[1], m.pos[2]]))  # Ensure position is a list

# Create meshes for fixes (if any)
fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=[f.pos[0], f.pos[1], f.pos[2]]))  # Set position of each fix

# Create spring connections (using lines)
springpos = []
for s in mss.springs:
    if isinstance(s.connectors[0], Mass):
        pA = s.connectors[0].pos
    else:
        pA = s.connectors[0].pos  # Position of Fix

    if isinstance(s.connectors[1], Mass):
        pB = s.connectors[1].pos
    else:
        pB = s.connectors[1].pos  # Position of Fix

    springpos.append([pA, pB])  # Add spring start and end positions

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')  # Line material with cyan color
springs = LineSegments2(springgeo, m2)

# Create axes for reference
axes = AxesHelper(1)  # Creates axes with size 1 for reference

# Set up the scene
scene = Scene(children=[*masses, *fixes, springs, axes])

# Set up camera and lighting
view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

controller = OrbitControls(controlling=camera)

# Renderer setup
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=view_width, height=view_height)

# Function to simulate the spinning top and update visualization
for i in range(10000):
    # Simulate for 0.02 seconds and 100 steps
    mss.simulate(0.02, 100)

    # Apply angular velocity to the masses (rotate them around the center)
    apply_angular_velocity(mss.masses, angular_velocity, 0.02)  # Apply spin at each step

    # Update positions of the masses in the visualization
    for m, mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])  # Update position

    # Update spring positions
    springpos = []
    for s in mss.springs:
        if isinstance(s.connectors[0], Mass):
            pA = s.connectors[0].pos
        else:
            pA = s.connectors[0].pos  # Position of Fix
        
        if isinstance(s.connectors[1], Mass):
            pB = s.connectors[1].pos
        else:
            pB = s.connectors[1].pos  # Position of Fix

        springpos.append([pA, pB])  # Add spring positions

    springs.geometry = LineSegmentsGeometry(positions=springpos)  # Update spring geometry

    # Render the scene
    renderer.render(scene, camera)

    sleep(0.01)  # Pause before the next frame

    # Optionally, stop after some steps for testing purposes
    if i == 1000:
        print("Simulation complete.")
        break  # Exit the loop for testing purposes


Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...


Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 0.02...
Simulating for 100 steps with time step 